<a href="https://colab.research.google.com/github/kd303/trnsfrmr_pytrch_end_p1/blob/main/session_5_torchtext/assignment/Assignment_DBPedia_torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from torchtext.datasets import DBpedia

In [6]:
help(DBpedia)

Help on function DBpedia in module torchtext.datasets.dbpedia:

DBpedia(root='.data', split=('train', 'test'))
    DBpedia dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 560000
    
        test: 70000
    
    
    Number of classes
        14
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [7]:
train_iter = DBpedia(split='train')

100%|██████████| 68.3M/68.3M [00:00<00:00, 71.9MB/s]


In [8]:
type(DBpedia(split='train'))

torchtext.data.datasets_utils._RawTextIterableDataset

In [9]:
next(train_iter)

(1,
 'E. D. Abbott Ltd  Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.')

In [10]:
for (line_number, (label, line)) in enumerate(train_iter):
  print(label, line)
  if line_number == 19:
    break

1 Schwan-Stabilo  Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss.
1 Q-workshop  Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished.
1 Marvell Software Solutions Israel  Marvell Software Solutions Israel known as RADLAN Computer Communications Limited before 2007 is a wholly owned subsidiary of Marvell Technology Group that specializes in local area network (LAN) technologies.
1 Bergan Mercy Medical Center

In [11]:
from torch.utils.data import DataLoader
train_iter = DBpedia(split = 'train')

help(DataLoader)

Help on class DataLoader in module torch.utils.data.dataloader:

class DataLoader(typing.Generic)
 |  DataLoader(*args, **kwds)
 |  
 |  Data loader. Combines a dataset and a sampler, and provides an iterable over
 |  the given dataset.
 |  
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with single- or multi-process loading, customizing
 |  loading order and optional automatic batching (collation) and memory pinning.
 |  
 |  See :py:mod:`torch.utils.data` documentation page for more details.
 |  
 |  Args:
 |      dataset (Dataset): dataset from which to load the data.
 |      batch_size (int, optional): how many samples per batch to load
 |          (default: ``1``).
 |      shuffle (bool, optional): set to ``True`` to have the data reshuffled
 |          at every epoch (default: ``False``).
 |      sampler (Sampler or Iterable, optional): defines the strategy to draw
 |          samples from the dataset. Can be any ``Iterable``

In [12]:
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False)

In [13]:
next(iter(dataloader))

[tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 ('E. D. Abbott Ltd  Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.',
  "Schwan-Stabilo  Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss.",
  'Q-workshop  Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website 

In [14]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [15]:
help(get_tokenizer)

Help on function get_tokenizer in module torchtext.data.utils:

get_tokenizer(tokenizer, language='en')
    Generate tokenizer function for a string sentence.
    
    Args:
        tokenizer: the name of tokenizer function. If None, it returns split()
            function, which splits the string sentence by space.
            If basic_english, it returns _basic_english_normalize() function,
            which normalize the string first and split by space. If a callable
            function, it will return the function. If a tokenizer library
            (e.g. spacy, moses, toktok, revtok, subword), it returns the
            corresponding library.
        language: Default en
    
    Examples:
        >>> import torchtext
        >>> from torchtext.data import get_tokenizer
        >>> tokenizer = get_tokenizer("basic_english")
        >>> tokens = tokenizer("You can now install TorchText using pip!")
        >>> tokens
        >>> ['you', 'can', 'now', 'install', 'torchtext', 'using

In [16]:
tokenizer = get_tokenizer("basic_english")

In [17]:
tokens = tokenizer("You can now install TorchText using pip!")
tokens

['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!']

In [18]:
help(build_vocab_from_iterator)

Help on function build_vocab_from_iterator in module torchtext.vocab.vocab_factory:

build_vocab_from_iterator(iterator: Iterable, min_freq: int = 1, specials: Union[List[str], NoneType] = None, special_first: bool = True) -> torchtext.vocab.vocab.Vocab
    Build a Vocab from an iterator.
    
    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        min_freq: The minimum frequency needed to include a token in the vocabulary.
        specials: Special symbols to add. The order of supplied tokens will be preserved.
        special_first: Indicates whether to insert symbols at the beginning or at the end.
    
    
    Returns:
        torchtext.vocab.Vocab: A `Vocab` object
    
    Examples:
        >>> #generating vocab from text file
        >>> import io
        >>> from torchtext.vocab import build_vocab_from_iterator
        >>> def yield_tokens(file_path):
        >>>     with io.open(file_path, encoding = 'utf-8') as f:
        >>> 

In [19]:
tokenizer = get_tokenizer("basic_english")
train_iter = DBpedia(split = 'train')

def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

In [20]:
def my_func():
  print("Hello! Welcome to the Dothraki Kingdon")
  yield
  print("Have you brought some Gold for us?")
  yield
  print("NO? Then you must die!!!")

In [21]:
temp_my_func = my_func()
next(temp_my_func)

Hello! Welcome to the Dothraki Kingdon


In [22]:
next(temp_my_func)

Have you brought some Gold for us?


In [23]:
## next(temp_my_func)

In [24]:
help(vocab)

Help on Vocab in module torchtext.vocab.vocab object:

class Vocab(torch.nn.modules.module.Module)
 |  Vocab(vocab)
 |  
 |  Base class for all neural network modules.
 |  
 |  Your models should also subclass this class.
 |  
 |  Modules can also contain other Modules, allowing to nest them in
 |  a tree structure. You can assign the submodules as regular attributes::
 |  
 |      import torch.nn as nn
 |      import torch.nn.functional as F
 |  
 |      class Model(nn.Module):
 |          def __init__(self):
 |              super(Model, self).__init__()
 |              self.conv1 = nn.Conv2d(1, 20, 5)
 |              self.conv2 = nn.Conv2d(20, 20, 5)
 |  
 |          def forward(self, x):
 |              x = F.relu(self.conv1(x))
 |              return F.relu(self.conv2(x))
 |  
 |  Submodules assigned in this way will be registered, and will have their
 |  parameters converted too when you call :meth:`to`, etc.
 |  
 |  :ivar training: Boolean represents whether this module is in tr

In [25]:
vocab.set_default_index(vocab["<unk>"])

In [26]:
vocab(['here', 'is', 'an', 'example', 'of', 'alien', 'invasion', 'they', 'are', 'called', 'bangalorites'])

[1538, 6, 19, 1840, 4, 5328, 4138, 168, 47, 304, 0]

In [27]:
vocab["<unk>"]

0

In [28]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [29]:
text_pipeline("Here is an exmaple of alient invasion, and they are called bangaloriters")

[1538, 6, 19, 0, 4, 0, 4138, 90515, 7, 168, 47, 304, 0]

In [30]:
label_pipeline('19')

18

In [31]:
from torch.utils.data import DataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def collate_fn(batch):
  src_batch, tgt_batch = [], []
  for src_batch, tgt_batch in batch:
    src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
    tgt_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
  src_batch = pad_sequences(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequences(tgt_batch, padding_value=PAD_IDX)
  return src_batch, tgt_batch

In [32]:
weight = torch.randn(10, 5)
weight

tensor([[ 1.5944, -0.8532, -0.1321, -0.0680,  0.1143],
        [-0.2533,  0.0119,  0.1340,  1.5424,  0.4331],
        [-2.2236,  2.2732,  1.5229,  0.1983, -0.4779],
        [-0.4710,  0.4432, -1.2667, -2.2129,  0.2228],
        [ 1.5305, -1.3368,  0.8037, -0.5994,  0.4910],
        [-0.5367,  1.5748,  0.2565,  0.8992, -0.4226],
        [ 1.1191,  0.7422,  0.3512,  0.8502,  0.2880],
        [ 0.1959, -0.8018, -1.4487, -0.4668, -0.4109],
        [ 1.3160, -0.1775, -0.1438,  0.4066, -1.7205],
        [-0.3187, -0.7105,  0.8181,  0.4216, -1.1274]])

In [33]:
indices = torch.tensor([4, 1, 7])
indices

tensor([4, 1, 7])

In [34]:
embeddings = torch.nn.functional.embedding(indices, weight)
embeddings

tensor([[ 1.5305, -1.3368,  0.8037, -0.5994,  0.4910],
        [-0.2533,  0.0119,  0.1340,  1.5424,  0.4331],
        [ 0.1959, -0.8018, -1.4487, -0.4668, -0.4109]])

In [35]:
embeddings.mean(dim=0, keepdim=True)

tensor([[ 0.4910, -0.7089, -0.1703,  0.1587,  0.1711]])

In [36]:
torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode="mean")

tensor([[ 0.4910, -0.7089, -0.1703,  0.1587,  0.1711]])

In [37]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [38]:
train_iter = DBpedia(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [39]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

1: World, 2: Sports, 3 Business, 4 Sci/Tech

In [47]:
train_iter = DBpedia(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [48]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [49]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = DBpedia()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 8313 batches | accuracy    0.697
| epoch   1 |  1000/ 8313 batches | accuracy    0.909
| epoch   1 |  1500/ 8313 batches | accuracy    0.938
| epoch   1 |  2000/ 8313 batches | accuracy    0.954
| epoch   1 |  2500/ 8313 batches | accuracy    0.958
| epoch   1 |  3000/ 8313 batches | accuracy    0.960
| epoch   1 |  3500/ 8313 batches | accuracy    0.965
| epoch   1 |  4000/ 8313 batches | accuracy    0.968
| epoch   1 |  4500/ 8313 batches | accuracy    0.967
| epoch   1 |  5000/ 8313 batches | accuracy    0.970
| epoch   1 |  5500/ 8313 batches | accuracy    0.969
| epoch   1 |  6000/ 8313 batches | accuracy    0.971
| epoch   1 |  6500/ 8313 batches | accuracy    0.974
| epoch   1 |  7000/ 8313 batches | accuracy    0.973
| epoch   1 |  7500/ 8313 batches | accuracy    0.973
| epoch   1 |  8000/ 8313 batches | accuracy    0.973
-----------------------------------------------------------
| end of epoch   1 | time: 52.93s | valid accuracy    0.975 
---------------

In [ ]:
!pip install wandb
import wandb
wandb.init()

In [43]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.981


This is a Business news
